# 通过Deepl Translator制作多语言epub格式的电子书

## 1. 基础介绍

### 1.1 为什么用.epub
.epub是最通用的电子书格式，所有格式的电子书都可以转换为.epub。推荐使用强大的[Calibre](https://calibre-ebook.com/)进行电子书管理和格式转换。Calibre支持直接转换为docx文件，可以直接丢到deepl去翻译，只不过docx文件不适合电子书阅读设配使用。我个人很少直接用电脑进行阅读。

亚马逊加密后的.azw3可以参考本文最后的脱壳方法转换成.epub。

### 1.2 .epub里面都有什么

.epub格式的电子书其实就是打包好的.html文件并加上一些metadata，和我们平时所浏览的网页一样。书本正文都储存在.html中。通过更改.epub后缀为.zip我们可以在直接浏览这些.html文件的内容。

根据ebooklib作者的回答（[Stack Overflow](https://stackoverflow.com/questions/66061399/modify-epub-file-by-pythons-ebooklib-but-all-the-contents-inside-head-was-lo)），应该无法直接修改.epub文件中.html的文字内容。因此我们通过解压.epub文件，替换.html文件再打包达到同样的效果。

### 1.3 .电子书翻译流程

因此整个翻译过程其实就是抓取.epub文件中.html中主题内容，翻译后再打包。


因此我们需要以上这些库的支持
>- 通过ebooklib解析.epub格式的电纸书，
>- 通过beautifulsoup4处理.html文件内容，
>- 再通过deepl api翻译内容。 ([申请deepl开发者账号，免费版可处理50w字节](https://www.deepl.com/pro-api?cta=header-pro)）

### 1.4 安装依赖

In [ ]:
!pip install ebooklib 
!pip install beautifulsoup4
!pip install deepl 
!pip install zipfile
!pip install lxml

## 2. Import 所需要的依赖

In [3]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import deepl
import zipfile
import shutil
from pathlib import Path

## 3. 使用Deepl 翻译

大部分文章主体都是存储于.html文件中‘p'的tag中，少数标题目录储存在'i'tag中，因此只翻译这两部分中的内容

In [8]:
def deepl_translate(soup, target_lang="ZH", trans_type=0):
    """
    :param target_lang:目标语言，ZH中文，JA日文，EN-US英文。更多语言支持可以参考deepl开发文档
    :param trans_type：生成样式，0为单翻译版本，1为双语对照版本
    :return:
    """
    for i in soup.findAll('p'):
        a = str(i.getText())
        if a == "":
            result = ""
        else:
            result = translator.translate_text(i.getText(), target_lang=target_lang)
            if trans_type == 1:
                i.string = i.text.replace(a, i.getText() + " \n" + result.text)
            elif trans_type == 0:
                i.string = i.text.replace(a, result.text)
            else:
                print("Error Translate Type")

    for i in soup.find_all('i', text=True):
        a = str(i.getText())
        if a == "":
            result = ""
        else:
            result = translator.translate_text(i.getText(), target_lang=target_lang)
            if trans_type == 1:
                i.string = i.text.replace(a, i.getText() + " \n" + result.text)
            elif trans_type == 0:
                i.string = i.text.replace(a, result.text)
            else:
                print("Error Translate Type")

## 4. 运行并生成
>- 需要有.epub电子书文件，放在books/epub/目录下。
>- 需要deepl 开发者Api key
>- 生成的.epub 电子书会存在books目录下

所有的目录都可以修改为自己想要的地方，所有路径最好不带有空格。

由于free account api只有**50w字符**，其实对整书翻译并不是特别友好。付费版为**25美元每100w字符**。

不追求epub格式的话其实可以使用docx格式上传翻译，不过deepl免费版本对于不同语言宽容度挺奇怪的。我免费版翻译日文7w字符的docx文件没什么问题，但是英文版2w字符就会报错文件过大。由于未知原因我无法直接购买pro版本，还在跟support team沟通，不知道pro版本有没有单docx文件字符的限制。如果没有限制，接下来会改为通过打包为docx文件的方式上传翻译，然后拆解.doxc文件内容还原为.epub。

In [ ]:
# 初始化deepl translator, 需要deepl开发者API Key
# example your_api = "4e324f8d-543s-22d3-32c3-a6497123ae76:fx"

your_api = ""
translator = deepl.Translator(your_api)

# 目标翻译语言
target_lang = "ZH"  # 目标语言，ZH中文，JA日文，EN-US英文。更多语言支持可以参考deepl开发文档
trans_type = 0  # trans_type：生成样式，0为单翻译版本，1为双语对照版本

# 书名，如果路径、书名中有空格，替换为'_'
book_name = 'One_Hundred_Years_of_Solitude'

# .epub电子书路径
book_path = 'books\epub\{}.epub'.format(book_name)

# 获取 所有.html文件内容
book = epub.read_epub(book_path)
items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))

# 解压.epub并储存于books/unzip_tmp
with zipfile.ZipFile(book_path, 'r') as zip_ref:
    zip_ref.extractall('books/unzip_tmp')

# 遍历所有.html并翻译文本
for item in items:
    soup = BeautifulSoup(item.get_body_content(), 'html.parser')
    fn = item.file_name
    deepl_translate(soup, target_lang=target_lang, trans_type=0)

    # 另存为新的.html文件
    with open("books/unzip_tmp/{}".format(fn), "wb") as f_output:
        f_output.write(soup.prettify("utf-8"))

# 重新打包,并输出到books目录下
shutil.make_archive('books/{}_{}'.format(book_name, trans_type), 'zip', 'books/unzip_tmp')
p = Path('books/{}_{}'.format(book_name, trans_type))
p.rename(p.with_suffix('.epub'))


## 额外资料
>- [将ammazon购买的加密过的.azw3格式的电子书转换为epub格式教程 （需要拥有kindle设备并有序列码）](https://www.7benshu.com/post/2019/11/21-1/)
>- [Calibre官网（电子书管理和格式转换软件）](https://calibre-ebook.com/) 

PS：有更好的方式可以联系我改进，vx：zixiaoyu9，mixin：37368406，email: yuzixiaobook@gmail.com